In [2]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd 
from tqdm import tqdm 
from sklearn.model_selection import train_test_split
from glob import glob

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

1.12.1+cu113
True
0
NVIDIA GeForce GTX 1080 Ti


In [ ]:
BATCH_SIZE = 1
MAX_LEN = 300 

category_name = '스포츠의류'
model_path = glob('model/model_{}/results/*.pt'.format(category_name))[-1]
model_name = "beomi/kcELECTRA-base"

model = torch.load(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_name)

device ='cpu'
model.to(device)

sentence 입력시 

'제품명' [제품명] 리뷰 [리뷰]  

이렇게 입력해주어야 더 정확도가 올라감.

In [ ]:
sentence = "제품명 고스트리퍼블릭 오버핏 자수 터틀 후드집업 GHZ-254, 2, 블랙 리뷰 입었을때 마치 남친 옷 뺏어입은거 같은 룩 입니다매우 편안하고 재질도 부들부들 하니 좋네요"
test = pd.DataFrame({'input':[sentence]})
test

In [ ]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, 'input']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

test_data = TestDataset(test)

test_dataloader = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
    )

In [ ]:
# inference
print('>> inference start <<')    
preds = [] 
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
          preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)